# BIOEE 4940 : **Introduction to Quantitative Analysis in Ecology**
### ***Spring 2021***
### Instructor: **Xiangtao Xu** ( ✉️ xx286@cornell.edu)
### Teaching Assistant: **Yanqiu (Autumn) Zhou** (✉️ yz399@cornell.edu)

---

## <span style="color:royalblue">Lab 2</span> *Spatial Data Analysis with Google Earth Engine*
*Adapted from [geemap.org](geemap.org)

### 0. Google Earth Engine (GEE) as a new tool for ecological analysis

**What is GEE?**
* A cloud-based platform for planetary scale geospatial analysis
* Uses Google's computational resources to reduce processing time
* A massive [archive](https://developers.google.com/earth-engine/datasets/catalog/) of remote sensing and other spatial data
* Time lapse [examples](https://earthengine.google.com/timelapse/)

GEE has a online code editor where you can use javascript to process, visualize, and download GEE data

<img src="https://github.com/xiangtaoxu/earthengine-community/raw/master/tutorials/beginners-cookbook/ee-editor.png" alt="GEE code editor" style="width: 600px;"/>

GEE also has an official set of API (Application Programming Interface) for python, and there are several additional packaged to better connect with GEE in python (R also has similar packages, e.g. [rgee](https://r-spatial.github.io/rgee/)). We are going to use geemap for our lab today.


**Configuration**

1. Python API installation ([more details here](https://developers.google.com/earth-engine/guides/python_install-conda))

    In your conda terminal run `conda install -c conda-forge earthengine-api`
    
2. GEEMAP installation ([more details here](https://geemap.org/installation/))

    In your conda terminal run `conda install -c conda-forge geemap`


**Learning Objectives of this lab:**

1. Explore data in GEE and visualize spatial data using python geemap
2. Process, and visualize, and Remote Sensing ImageCollections
3. Export data and plots
---

#### 1. GEE initialization and basic data visualization using geemap

In [ ]:
# first always need to import ee (earth engine api) and initialize

import ee

try:
    ee.Initialize()
except:
    # need to authenticate with your credential at the first time
    ee.Authenticate()
    ee.Initialize()

In [ ]:
#import geemap 
import geemap.eefolium as geemap
# theoretically, we should just import geemap directly
# However, the default geemap uses ipyleaflet for interactive map visualization, which does not work for my environment...
# Here, we use the folium-based interactive mapping
# ipyleaflet can allow for more interactive features


In [ ]:
# lat and lon for Corson Hall, where this course should be happening...
corson_loc = [42.447, -76.479]

# initialize an interactive map based on google map, centered at center
Map = geemap.Map(center=corson_loc, zoom=20)

#optional changes in basemap
#Map.add_basemap('HYBRID')

#Display the map
Map

In [ ]:
# show an elevational map for Tompkins County
# https://developers.google.com/earth-engine/datasets/catalog/USGS_NED

# define the image source
elevationImage = ee.Image('USGS/NED').select('elevation')

# parameters for visualization
elevationVis = {
  'min': 0.0,
  'max': 750.0,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
};

# zoom out a little to see the whole Ithaca
Map = geemap.Map(center=corson_loc, zoom=10)

# add the image to the map
Map.addLayer(elevationImage, elevationVis, 'Elevation');

# outline tompkins county
counties = ee.FeatureCollection('TIGER/2016/Counties')
tompkins = ee.Feature(counties.filter(
    ee.Filter.eq('NAME', 'Tompkins')).first())
Map.addLayer(ee.Image().paint(tompkins, 0, 2), {
             'palette': 'yellow'}, 'Tompkins')


# add colorbar
colors = elevationVis['palette']
vmin = elevationVis['min']
vmax = elevationVis['max']

Map.add_colorbar(colors=colors, vmin=vmin, vmax=vmax)

Map.addLayerControl()

Map

In [ ]:
## Mean Annual Temperature Map
# https://developers.google.com/earth-engine/datasets/catalog/WORLDCLIM_V1_BIO

# define the image source
# divide the scale of the data
Image = ee.Image('WORLDCLIM/V1/BIO').select('bio01').divide(10.)

# parameters for visualization
VisParams = {
  'min': -23.0,
  'max': 30.0,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
};

# zoom out a little to see the whole Ithaca
Map = geemap.Map(center=corson_loc, zoom=5)

# add the image to the map
Map.addLayer(Image, VisParams, 'MAT');

# outline tompkins county
counties = ee.FeatureCollection('TIGER/2016/Counties')
tompkins = ee.Feature(counties.filter(
    ee.Filter.eq('NAME', 'Tompkins')).first())
Map.addLayer(ee.Image().paint(tompkins, 0, 2), {
             'palette': 'yellow'}, 'Tompkins')


# add colorbar
colors = VisParams['palette']
vmin = VisParams['min']
vmax = VisParams['max']

Map.add_colorbar(colors=colors, vmin=vmin, vmax=vmax)

Map.addLayerControl()

Map


In [ ]:
## land cover map
# define the image source
# divide the scale of the data
landcover = ee.Image('USGS/NLCD/NLCD2016').select('landcover')


# zoom out a little to see the whole Ithaca
Map = geemap.Map(center=corson_loc, zoom=15)

# add the image to the map
Map.addLayer(landcover, {}, 'Land Cover')

# outline tompkins county
counties = ee.FeatureCollection('TIGER/2016/Counties')
tompkins = ee.Feature(counties.filter(
    ee.Filter.eq('NAME', 'Tompkins')).first())
Map.addLayer(ee.Image().paint(tompkins, 0, 2), {
             'palette': 'yellow'}, 'Tompkins')


# add colorbar
Map.add_legend(builtin_legend='NLCD')

Map.addLayerControl()

Map

#### 2. Visualize remote sensing Image Collections

In [ ]:
# overlay observations from Sentinel-2 satellite (10m resolution) around Corson Hall

# initialize an interactive map based on google map, centered at center
Map = geemap.Map(center=corson_loc, zoom=20)

# Load an image, will show how to identify this image later....
image = ee.Image('COPERNICUS/S2_SR/20200922T154859_20200922T160118_T18TUN')

# Define visualization parameters in an object literal.
vizParams = {'bands': ['B4', 'B3', 'B2'],
             'min': 0, 'max': 3000}

# Display the image.
Map.addLayer(image, vizParams, 'Sentinel 2 original image')

Map.addLayerControl()


# Display the map.
Map

In [ ]:
# Plot NDVI, normalized difference vegetation index
# (NIR - Red) / (NIR + Red)
Map = geemap.Map(center=corson_loc, zoom=20)

ndvi = image.normalizedDifference(['B8', 'B4'])
ndviViz = {'min': 0.0, 'max': 0.75, 'palette': ['FF0000', '008000']}


# Display the image.
Map.addLayer(ndvi, ndviViz, 'Sentinel 2 NDVI')



# add colorbar
colors = ndviViz['palette']
vmin = ndviViz['min']
vmax = ndviViz['max']

Map.add_colorbar(colors=colors, vmin=vmin, vmax=vmax)

Map.addLayerControl()

# Display the map.
Map

In [ ]:
# Plot NDVI, normalized difference water index
# For water body detection
# (Green - NIR) / (Green + NIR)
Map = geemap.Map(center=corson_loc, zoom=15)

ndwi = image.normalizedDifference(['B3', 'B8'])
ndwiViz = {'min': 0.0, 'max': 0.5, 'palette': ['808000', '00FFFF']}


# Display the image.
Map.addLayer(ndwi, ndwiViz, 'Sentinel 2 NDWI')



# add colorbar
colors = ndwiViz['palette']
vmin = ndwiViz['min']
vmax = ndwiViz['max']

Map.add_colorbar(colors=colors, vmin=vmin, vmax=vmax)

Map.addLayerControl()


# Display the map.
Map


#### 3. ImageCollections and Data/Image Export

In [ ]:
# Let's work with Arnot Forest this time
af_loc = [42.271, -76.628] # a point within Arnot Forest

# identify a 500 meter buffer around our Point Of Interest (POI)
poi = ee.Geometry.Point(af_loc[1],af_loc[0]).buffer(500)

# get MODIS Enhanced Vegetation Index (EVI) MOD13Q1.006
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13Q1
    
collection = (
    ee.ImageCollection('MODIS/006/MOD13Q1')   # name of image collection
      .filterDate('2019-05-01', '2019-11-01') # filter by time, get growing season in 2019
      .filterBounds(poi)                      # filter by location
      #.filterMetadata() # filter by meta-data , e.g. data quality
            )

print(collection.size().getInfo())
collection.aggregate_array('system:id').getInfo()

In [ ]:
# reduce the map into a single growing season average EVI
Map = geemap.Map(center=corson_loc, zoom=10)


# outline tompkins county
counties = ee.FeatureCollection('TIGER/2016/Counties')
tompkins = ee.Feature(counties.filter(
    ee.Filter.eq('NAME', 'Tompkins')).first())

meanNDVI = collection.select('EVI').mean().clip(tompkins)

visPar={'min': 0.0, 'max': 0.75*10000, 'palette': ['FF0000', '008000']}

Map.addLayer(meanNDVI, visPar, 'Mean 2019 EVI May-Oct')

# Display Map
Map

In [ ]:
geemap.ee_export_image(meanNDVI, './Data/tompkins_ndvi.tif', scale=250, file_per_band=False)
# can also export to your G-drive using GEE Python API

In [ ]:
# plot the map with rasterio
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import rasterio
import matplotlib.pyplot as plt
%matplotlib inline


filename = './data/tompkins_ndvi.tif'
src = rasterio.open(filename, 'r')

# read image into ndarray
im = src.read()

# transpose the array from (band, row, col) to (row, col, band)
im = np.transpose(im, [1,2,0])

# calculate extent of raster
# transform to cartopy format, left, right, low, up
img_extent = [src.bounds[0],src.bounds[2],src.bounds[1],src.bounds[3]]

In [ ]:
# define cartopy crs for the raster, based on rasterio metadata
crs = ccrs.PlateCarree()

# create figure
fig = plt.figure()
ax = plt.axes(projection=crs)
plt.title('Tompkins Mean NDVI')
#ax.set_xmargin(0.05)
#ax.set_ymargin(0.10)

# plot raster
him = plt.imshow(im / 10000., origin='upper',extent=img_extent, transform=crs, interpolation='nearest')

# add grid lines
hg = ax.gridlines(draw_labels=True)
hg.right_labels = False

plt.colorbar(him,ax=ax, shrink=.5,label='NDVI')

plt.show()

# can also use cartoee in geemap


In [ ]:
# export image collections

# use map function to batch process image collection
# use lambda function in python

EVI_to_export=collection.select('EVI').map(lambda img: img.clip(tompkins))
geemap.ee_export_image_collection(EVI_to_export, './data/', scale=250, file_per_band=False)


In [ ]:
# save time-series
collection = (
    ee.ImageCollection('MODIS/006/MOD13Q1')   # name of image collection
      .filterDate('2010-01-01', '2020-01-01') # filter by time
      .filterBounds(poi)                      # filter by location
            )

def poi_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=poi, scale=250).get('EVI')
    return img.set('date', img.date().format()).set('mean',mean)

poi_reduced_imgs = collection.map(poi_mean)

nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)

import pandas as pd
df = pd.DataFrame(nested_list.getInfo(), columns=['date','mean'])
df.to_csv('./data/arnot_forest_evi.csv')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df.plot(x='date',y='mean')

Miscellaneous

In [ ]:
# get an geometry object for Tompkins County

counties = ee.FeatureCollection('TIGER/2016/Counties')
tompkins = ee.Feature(counties.filter(
    ee.Filter.eq('NAME', 'Tompkins')).first())
roi = ee.Geometry.Point(corson_loc[1],corson_loc[0]) # lon and lat

# get the one with least cloud
S2 = (
    ee.ImageCollection('COPERNICUS/S2_SR')
      .filterDate('2018-01-01', '2021-01-01')
      .filterBounds(roi)
      .sort("CLOUD_COVERAGE_ASSESSMENT")
      .first()
     )

# get image name
print(S2.get("system:id").getInfo())

In [ ]:
counties = ee.FeatureCollection('TIGER/2016/Counties')
tompkins = ee.Feature(counties.filter(
    ee.Filter.eq('NAME', 'Tompkins')).first())
roi = ee.Geometry.Point(corson_loc[1],corson_loc[0]) # lon and lat
S2 = (
    ee.ImageCollection('COPERNICUS/S2_SR')
      .filterDate('2018-01-01', '2021-01-01')
      .filterBounds(roi)
      .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',25.)
     )
print(S2.size().getInfo())